#### Task 3: 
#### Scrape data for all products from the Men's Clearance section using Python from https://www.next.co.uk/ . The data we want you to collect should be for United Kingdom. Make sure that you select the right country and the prices are in GB Pounds (£).
>The output should be a .csv file with the following column headers.  
>Name: the name or the title of the product   
>Brand: the brand name of the product   
>Original Price: the price of the product before the discount   
>Discounted Price: the price of the product after the discount   
>Image Url: the url of the product image 

#### Import All Necessary packages

In [49]:
#import all necessary packages
from bs4 import BeautifulSoup
import re, csv, time, random, pandas as pd
from lxml import html
import requests
import math
from csv import writer
from selenium import webdriver
import urllib.request
from lxml import html
from selenium.webdriver.support.wait import WebDriverWait

#### All the xpaths which will be used to extract  the particular data through the crawling

In [50]:
#Configuration Block 
Product_Numbers = '/html/body/section[1]/section[1]/div/div[7]/div[1]'
Product_Name_Xpath = './div[2]/div[1]/div[1]/text()'
Item_Number_Xpath = './div[2]/div[1]/div[2]/text()'
Original_Price_Xpath = './div[2]/div[3]/div[1]/text()'
Discounted_Price_Xpath = './div[2]/div[3]/div[2]/div[2]/text()'
Image_Url_Xpath = './div[1]/a[2]/div/img/@src'
Product_Number_Per_Page_Xapth = '//div[@class="search-result-item multiple"]'

#### *Heading 1: Start Crawling for the products from United Kingdom with price in  GB Pounds (£).

In [57]:
#Provide the Web Url
url = "https://www.next.co.uk/clearance/search?w=*&af=gender:men%20gender:men%20&srt=24#searchpos_1_24"
#Use Mozilla Firefox Web selenium driever
driver = webdriver.Firefox()
#Give time to load the web page to fetch the complete content
driver.set_page_load_timeout(100) 
#start automation
driver.get(url)
#Wait for the Page loading
time.sleep(2) 
driver.implicitly_wait(1000)
#maximize the browser
driver.maximize_window()

#Get lxml trees from the raw data
HTMLTree = html.fromstring(driver.page_source)

#Get The Number of Products details
NProducts = HTMLTree.xpath(Product_Numbers)
total_prodcuts = int(NProducts[0].text_content().split()[2].replace(',',''))
print('Total Number of Products Available : ' , total_prodcuts)

#Crawl products per page
per_page_products = 24
total_pages_needs_to_be_crawled = math.ceil(total_prodcuts/per_page_products)
print('Total Pages needs to be crawled : ' , total_pages_needs_to_be_crawled)


output_rows = []

class NextCoUk:
    def __init__(self):
        self.start = 'start'
        
    def ManClearance(self,data,output_rows):
        for product_n in range(len(data)):
            product_name = data[product_n].xpath(Product_Name_Xpath)[0]
            item_number = data[product_n].xpath(Item_Number_Xpath)[0]
            product_original_price = data[product_n].xpath(Original_Price_Xpath)[0].strip().replace('Original','')
            product_discount_price = data[product_n].xpath(Discounted_Price_Xpath)[0]
            image_url = data[product_n].xpath(Image_Url_Xpath)[0]
            output_rows+= [[product_name,item_number,product_original_price,product_discount_price,image_url]]
        return output_rows
    
    def FetchDetails(self,driver):
        tree = html.fromstring(driver.page_source)
        data = tree.xpath(Product_Number_Per_Page_Xapth)
        return data

    
data = HTMLTree.xpath(Product_Number_Per_Page_Xapth)
#Create the object of the class and call a method
First_Page_Data = NextCoUk()
output_rows = First_Page_Data.ManClearance(data,output_rows)

#Base Url
base_url = "https://www.next.co.uk/clearance/search?w=*&af=gender:men%20gender:men%20&srt=24#searchpos_{}_{}"

#Crawl Product Details for Other Pages
for page_n in range(2 , total_pages_needs_to_be_crawled+1):
    rebuild_url = base_url.format(page_n,((page_n-1)*24) + 12 )
    driver.get(rebuild_url)
    #Wait for the page loading
    time.sleep(2)
    driver.implicitly_wait(1000)
    #maximize the browser
    driver.maximize_window()
    data = NextCoUk().FetchDetails(driver)
    print('Product Crawled from Page : ' , page_n ,' : ',  len(data))
    output_rows = NextCoUk().ManClearance(data[:24],output_rows)

driver.quit()
print('{} Products Have been crawled'.format(len(output_rows)))


Total Number of Products Available :  2228
Total Pages needs to be crawled :  93
Product Crawled from Page :  2  :  24
Product Crawled from Page :  3  :  24
Product Crawled from Page :  4  :  24
Product Crawled from Page :  5  :  24
Product Crawled from Page :  6  :  24
Product Crawled from Page :  7  :  24
Product Crawled from Page :  8  :  24
Product Crawled from Page :  9  :  24
Product Crawled from Page :  10  :  24
Product Crawled from Page :  11  :  24
Product Crawled from Page :  12  :  24
Product Crawled from Page :  13  :  24
Product Crawled from Page :  14  :  24
Product Crawled from Page :  15  :  24
Product Crawled from Page :  16  :  24
Product Crawled from Page :  17  :  24
Product Crawled from Page :  18  :  24
Product Crawled from Page :  19  :  24
Product Crawled from Page :  20  :  24
Product Crawled from Page :  21  :  24
Product Crawled from Page :  22  :  24
Product Crawled from Page :  23  :  24
Product Crawled from Page :  24  :  24
Product Crawled from Page :  2

#### Store Crawled information in excel file (.csv format) 

In [58]:
#create a list of column names 
Column_Names = [['Product_Name' ,'Product_Item_Number' , 'Original_Price' , 'Discount_Price' ,  'Image_Url']]
#append columns with data as header
output_rows = Column_Names + output_rows

#Write complete data in csv file 
#create excel data file
Filename = "NextCoUk.csv"  #Give file name with path
with open(Filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)

#### Read Dataset 

In [60]:
import pandas as pd
df = pd.read_csv(r'NextCoUk.csv', engine='python')
print('Length of dataset : ' , len(df))
print('Number of Duplicate Rows : ', df.duplicated().sum())
#Extract first 5 rows from the dataset
df.head()

Length of dataset :  2228
Number of Duplicate Rows :  0


,Product_Name,Product_Item_Number,Original_Price,Discount_Price,Image_Url
0,Bright Blue Slim Fit Check Suit: Jacket,361-381,£84,£16,https://xcdn.next.co.uk/COMMON/Items/Default/D...
1,Bright Blue Slim Fit Check Suit: Trousers,872-134,£45,£13.50,https://xcdn.next.co.uk/COMMON/Items/Default/D...
2,Under Armour Charged Trainers,976-269,£65,£32,https://xcdn.next.co.uk/COMMON/Items/Default/D...
3,Blue Regular Fit Linen Blend Long Sleeve Shirt,296-495,£26,£9,https://xcdn.next.co.uk/COMMON/Items/Default/D...
4,Grey Camouflage Cargo Shorts,205-660,£24,£12,https://xcdn.next.co.uk/COMMON/Items/Default/D...


#### Simple Data Analysis

In [63]:
df.describe()

,Product_Name,Product_Item_Number,Original_Price,Discount_Price,Image_Url
count,2228,2228,2140,2228,2228
unique,2148,2228,146,89,2228
top,Textured Suit: Jacket,279-840,£25,£15,https://xcdn.next.co.uk/COMMON/Items/Default/D...
freq,4,1,147,245,1


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2228 entries, 0 to 2227
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Product_Name         2228 non-null   object
 1   Product_Item_Number  2228 non-null   object
 2   Original_Price       2140 non-null   object
 3   Discount_Price       2228 non-null   object
 4   Image_Url            2228 non-null   object
dtypes: object(5)
memory usage: 87.2+ KB


In [61]:
#Check duplicate item
df[df.duplicated(['Product_Item_Number'])]

,Product_Name,Product_Item_Number,Original_Price,Discount_Price,Image_Url


#### Hence there is no duplicate things